In [72]:
import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import data_feeder as df
import numpy



class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        pretrained = False;
        if (pretrained == True):
            self.model = torchvision.models.vgg16(pretrained=True)
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.fc = nn.Linear(128, 1) 
        else:
            self.model = nn.Sequential(
                    nn.Conv2d(3, 8, 5, stride = 2) 
                    ,nn.BatchNorm2d(8)
                        ,nn.ReLU()
                        ,nn.Conv2d(8, 16, 5, stride = 2)
                        ,nn.BatchNorm2d(16)
                            ,nn.ReLU()
                        ,nn.Conv2d(16, 32, 5)
                            ,nn.BatchNorm2d(32)
                            ,nn.ReLU()
                            ,nn.Conv2d(32, 48, 5)
                            ,nn.BatchNorm2d(48)
                            ,nn.ReLU()
                            ,nn.Conv2d(48, 1, 5)
                )
            
           # self.fc1 = nn.Linear(24, 64)
           # self.fc2 = nn.Linear(64, 1)
              #                  ,nn.Linear(48 * 48 * 24, 128)
             #                       ,nn.BatchNorm1d(128)
            #                        ,nn.ReLU()
           #                         ,nn.Linear(128, 1)
                
    
    def forward(self, input_data):
        return self.model(input_data)
    
 #   def _bias_init(self, b):
 #       b.bias = torch.nn.Parameter(torch.zeros(len(b.bias)))

 #   def _weight_init(self, m):
  #      size = m.weight.size()
   #     fan_out = size[0]  # number of rows
    #    fan_in = size[1]  # number of columns
     #   variance = np.sqrt(2.0 / (fan_in + fan_out))
      #  m.weight.data.normal_(0.0, variance)

    
class Solver():
    def __init__(self):
        super(Solver, self).__init__()
        
    def loss(self, pred, labels):
        return (pred - labels).pow(2).sum() / len(labels)

    def train(self, train_data, num_epochs = 15, learning_rate = 1e-1, batch_size = 10, pretrained = False):

            train_loader = torch.utils.data.DataLoader(dataset = train_data,

                                               batch_size=batch_size, 

                                               shuffle=True)
            print(len(train_loader))
            cnn = Model()
            optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)

            for i in range(num_epochs):
                for j, (images, labels) in enumerate(train_loader):
                    images = Variable(images.float())
                    labels = Variable(labels.float())
                    optimizer.zero_grad()
                    pred = cnn(images)
                    loss = self.loss(pred, labels)
                    cnn.zero_grad()
                    loss.backward()
                    optimizer.step()
                       
                    if (j+1) % 10 == 0:

                        print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 

                           %(i+1, num_epochs, j+1, len(train_loader)//batch_size, loss.data[0]))
                        
train_data, val_data = df.getDrivingData('race1516207298.txt')

solver = Solver()
solver.train(train_data)         
    
        
    
    
    
    

40
Epoch [1/15], Iter [10/4] Loss: 50.5409
Epoch [1/15], Iter [20/4] Loss: 28.8200
Epoch [1/15], Iter [30/4] Loss: 7.2530
Epoch [1/15], Iter [40/4] Loss: 0.3962
Epoch [2/15], Iter [10/4] Loss: 0.9896
Epoch [2/15], Iter [20/4] Loss: 0.2479
Epoch [2/15], Iter [30/4] Loss: 0.0977
Epoch [2/15], Iter [40/4] Loss: 0.0685
Epoch [3/15], Iter [10/4] Loss: 0.0350
Epoch [3/15], Iter [20/4] Loss: 0.0940
Epoch [3/15], Iter [30/4] Loss: 0.0625
Epoch [3/15], Iter [40/4] Loss: 0.0090
Epoch [4/15], Iter [10/4] Loss: 0.0280
Epoch [4/15], Iter [20/4] Loss: 0.0367
Epoch [4/15], Iter [30/4] Loss: 0.0378
Epoch [4/15], Iter [40/4] Loss: 0.0141
Epoch [5/15], Iter [10/4] Loss: 0.0246
Epoch [5/15], Iter [20/4] Loss: 0.0369
Epoch [5/15], Iter [30/4] Loss: 0.0453
Epoch [5/15], Iter [40/4] Loss: 0.0143
Epoch [6/15], Iter [10/4] Loss: 0.0545
Epoch [6/15], Iter [20/4] Loss: 0.0758
Epoch [6/15], Iter [30/4] Loss: 0.0298
Epoch [6/15], Iter [40/4] Loss: 0.0184
Epoch [7/15], Iter [10/4] Loss: 0.0597
Epoch [7/15], Iter [